Word Embeddings: Encoding Lexical Semantics

# Import Modules

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

# Word Embeddings in Pytorch

In [16]:
word_to_ix = {"hello": 0, "world": 1}
embeds = nn.Embedding(2, 5)
print(type(embeds))
print(embeds)
print()

lookup_tensor = torch.tensor([word_to_ix["hello"]], dtype=torch.long)
print(word_to_ix["hello"])
print(lookup_tensor)
print()

hello_embed = embeds(lookup_tensor)
print(hello_embed)

<class 'torch.nn.modules.sparse.Embedding'>
Embedding(2, 5)

0
tensor([0])

tensor([[ 0.4728,  1.0049, -0.2871, -1.1619,  0.0276]], grad_fn=<EmbeddingBackward>)


# An Example: N-Gram Language Modeling

P(w_i|w_i-1, w_i-2, ..., w_i-n+1) を求める

In [26]:
CONTEXT_SIZE = 2
EMBEDDING_DIM = 10

test_sentence = """When forty winters shall besiege thy brow,
And dig deep trenches in thy beauty's field,
Thy youth's proud livery so gazed on now,
Will be a totter'd weed of small worth held:
Then being asked, where all thy beauty lies,
Where all the treasure of thy lusty days;
To say, within thine own deep sunken eyes,
Were an all-eating shame, and thriftless praise.
How much more praise deserv'd thy beauty's use,
If thou couldst answer 'This fair child of mine
Shall sum my count, and make my old excuse,'
Proving his beauty by succession thine!
This were to be new made when thou art old,
And see thy blood warm when thou feel'st it cold.""".split()
print(test_sentence)

['When', 'forty', 'winters', 'shall', 'besiege', 'thy', 'brow,', 'And', 'dig', 'deep', 'trenches', 'in', 'thy', "beauty's", 'field,', 'Thy', "youth's", 'proud', 'livery', 'so', 'gazed', 'on', 'now,', 'Will', 'be', 'a', "totter'd", 'weed', 'of', 'small', 'worth', 'held:', 'Then', 'being', 'asked,', 'where', 'all', 'thy', 'beauty', 'lies,', 'Where', 'all', 'the', 'treasure', 'of', 'thy', 'lusty', 'days;', 'To', 'say,', 'within', 'thine', 'own', 'deep', 'sunken', 'eyes,', 'Were', 'an', 'all-eating', 'shame,', 'and', 'thriftless', 'praise.', 'How', 'much', 'more', 'praise', "deserv'd", 'thy', "beauty's", 'use,', 'If', 'thou', 'couldst', 'answer', "'This", 'fair', 'child', 'of', 'mine', 'Shall', 'sum', 'my', 'count,', 'and', 'make', 'my', 'old', "excuse,'", 'Proving', 'his', 'beauty', 'by', 'succession', 'thine!', 'This', 'were', 'to', 'be', 'new', 'made', 'when', 'thou', 'art', 'old,', 'And', 'see', 'thy', 'blood', 'warm', 'when', 'thou', "feel'st", 'it', 'cold.']


In [27]:
trigrams = [([test_sentence[i], test_sentence[i + 1]], test_sentence[i + 2])
                     for i in range(len(test_sentence) - 2)]
print(trigrams[:3])

[(['When', 'forty'], 'winters'), (['forty', 'winters'], 'shall'), (['winters', 'shall'], 'besiege')]


In [28]:
vocab = set(test_sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

print(vocab)
print(word_to_ix)

{'all-eating', 'much', 'lusty', 'deep', 'Where', 'To', 'Proving', "youth's", 'dig', 'count,', 'own', 'answer', 'held:', 'This', 'to', 'days;', 'be', 'Shall', 'trenches', "totter'd", 'more', 'shall', 'fair', 'being', 'Will', 'now,', 'Then', 'of', 'sum', 'in', "excuse,'", "'This", 'see', 'so', 'weed', 'thou', 'made', 'cold.', 'And', 'treasure', 'thine', 'new', 'Were', 'the', 'were', 'beauty', 'worth', 'field,', 'praise.', 'Thy', 'by', 'blood', 'old,', 'his', 'asked,', 'mine', 'a', 'my', 'thriftless', 'thy', 'couldst', "feel'st", 'on', 'an', 'besiege', 'thine!', 'small', 'If', 'within', 'make', 'eyes,', 'proud', 'lies,', 'and', 'sunken', 'gazed', 'say,', 'brow,', 'How', 'use,', 'old', 'shame,', 'winters', "beauty's", 'livery', 'praise', 'it', 'art', 'When', "deserv'd", 'warm', 'forty', 'where', 'when', 'all', 'succession', 'child'}
{'all-eating': 0, 'much': 1, 'lusty': 2, 'deep': 3, 'Where': 4, 'To': 5, 'Proving': 6, "youth's": 7, 'dig': 8, 'count,': 9, 'own': 10, 'answer': 11, 'held:': 1

In [32]:
class NGramLanguageModeler(nn.Module):

    def __init__(self, vocab_size, embedding_dim, context_size):
        """
        Args:
            vocab_size (int): 語彙数
            embedding_dim (int): Embedding ベクトルの次元
            context_size (int): 
        """
        super(NGramLanguageModeler, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.linear1 = nn.Linear(context_size * embedding_dim, 128)
        self.linear2 = nn.Linear(128, vocab_size)
        
    def forward(self, inputs):
        embeds = self.embeddings(inputs).view((1, -1))  # Embedding を 語彙数 * Embeddingの次元 に view で引き伸ばしている
        out = F.relu(self.linear1(embeds))
        out = self.linear2(out)
        log_probs = F.log_softmax(out, dim=1)
        return log_probs

In [ ]:
losses = []
loss_function = nn.NLLLoss()  # Negative log-likelihood loss
model = NGramLanguageModeler(len(vocab), EMBEDDING_DIM, CONTEXT_SIZE)
optimizer = optim.SGD(model.parameters(), lr=0.001)

for epoch in range(10):
    total_loss = 0
    for context, target in trigrams:

        # context に含まれる単語からなる tensor を作成
        context_idxs = torch.tensor([word_to_ix[w] for w in context], dtype=torch.long)
        
        model.zero_grad()
        
        log_probs = model(context_idxs)
        
        target_tensor = torch.tensor([word_to_ix[target]], dtype=torch.long)
        loss = loss_function(log_probs, target_tensor)
        
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    losses.append(total_loss)
print(losses)

# Exercise: Computing Word Embeddings: Continuous Bag-of-Words

In [39]:
CONTEXT_SIZE = 2  # 2 words to the left, 2 to the right
raw_text = """We are about to study the idea of a computational process.
Computational processes are abstract beings that inhabit computers.
As they evolve, processes manipulate other abstract things called data.
The evolution of a process is directed by a pattern of rules
called a program. People create programs to direct processes. In effect,
we conjure the spirits of the computer with our spells.""".split()

In [40]:
vocab = set(raw_text)
vocab_size = len(vocab)

word_to_ix = {word: i for i, word in enumerate(vocab)}
data = []
for i in range(2, len(raw_text) - 2):
    context = [raw_text[i - 2], raw_text[i - 1],
                       raw_text[i + 1], raw_text[i + 2]]
    target = raw_text[i]
    data.append((context, target))
print(data[:5])

[(['We', 'are', 'to', 'study'], 'about'), (['are', 'about', 'study', 'the'], 'to'), (['about', 'to', 'the', 'idea'], 'study'), (['to', 'study', 'idea', 'of'], 'the'), (['study', 'the', 'of', 'a'], 'idea')]


TODO: 自分で CBOW 実装しなさい

In [42]:
class CBOW(nn.Module):
    
    def __init__(self):
        pass
    
    def forward(self, inputs):
        pass


def make_context_vector(context, word_to_ix):
    idxs = [word_to_ix[w] for w in context]
    return torch.tensor(idxs, dtype=torch.long)

make_context_vector(data[0][0], word_to_ix)

tensor([29, 26, 21, 33])